In [ ]:
import os
import time
import json
import sys
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer, util
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from tqdm import tqdm
from typing import TypedDict, Annotated
sys.path.insert(0, str(Path().resolve().parent / "app"))

In [ ]:
# Load environment
print("Chargement de l'environnement...")
load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
json_in = Path().resolve()/ "QA_pairs.json"
index_path = Path().resolve().parent / "document_index.json"
faiss_path = Path().resolve().parent / "faiss_index"

In [ ]:
# Choose if ypu want to use RAG or not
use_rag = True

In [ ]:
print("Chargement des composants du chatbot...")
model = ChatMistralAI(mistral_api_key=api_key, model="mistral-large-latest")
embedding_fn = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=api_key)
vector = FAISS.load_local(faiss_path, embeddings=embedding_fn, allow_dangerous_deserialization=True)

if use_rag:
    system_instruction_rag = """
    Tu es l’assistant de l’observatoire astronomique de l’IMT Atlantique (campus de Brest).
    Règles impératives :
    1. Réponds en français, en **trois phrases maximum**.
    2. Appuie-toi *exclusivement* sur le CONTEXTE fourni ; si l’info n’y est pas, réponds : « Je ne sais pas. »
    3. Toute formule → encadrée par des dollars : $\,E = mc^2\,$.
    4. Tu donne des réponses **précises** et **concises**.
    CONTEXTE :
    {context}
    """

    prompt_rag = ChatPromptTemplate.from_messages([
        ("system", system_instruction_rag.strip()),
        ("human", "{input}")
    ])

    retriever  = vector.as_retriever()

    document_chain = create_stuff_documents_chain(model, prompt_rag)
    chain_rag = create_retrieval_chain(retriever, document_chain)

    def rag_get_response(question, chain=chain_rag):
        while True :
            try:
                response = chain.invoke({"input": question})
                break
            except Exception as e:
                print(f"Erreur lors de l'appel à l'API : {e}")
        return response['answer']
else :
    system_instruction_norag = """
    Tu es l’assistant de l’observatoire astronomique de l’IMT Atlantique (campus de Brest).
    Tu ne disposes pas des documents complets ; réponds uniquement sur la base de tes connaissances internes.
    Contraintes :
    1. Français, trois phrases maximum.
    2. Formules mathématiques encadrées par des dollars (LaTeX).
    3. Ne cite aucune source.
    4. Si l’information précise n’est pas certaine, dis : « Je ne sais pas. »
    5. Tu donne des réponses **précises** et **concises**.
    """

    prompt_norag = ChatPromptTemplate.from_messages([
        ("system", system_instruction_norag.strip()),
        ("human", "{input}")
    ])
    output_parser = StrOutputParser()
    chain_norag = prompt_norag | model | output_parser

    def no_rag_get_response(question, chain=chain_norag):
        while True :
            try:
                response = chain.invoke({"input": question})
                break
            except Exception as e:
                print(f"Erreur lors de l'appel à l'API : {e}")
        return response

In [ ]:
print("Chargement des paires QA depuis QA_pairs.json...")
with open(index_path, encoding="utf-8") as f:
    doc_index = json.load(f)             # doc_name → UUID

with open(json_in, encoding="utf-8") as f:
    raw_pairs = json.load(f)

pairs = []
for p in raw_pairs:
    if not p.get("support_doc_ids"):
        uuid_ = doc_index.get(p["doc_name"])
        p["support_doc_ids"] = [uuid_] if uuid_ else []
    pairs.append(p)

lookup = {p["question"]: p for p in pairs}


In [ ]:
csv_name = "rag_eval_results.csv" if use_rag else "no_rag_eval_results.csv"

csv  = Path().resolve() / csv_name
if os.path.exists(csv):
    df = pd.read_csv(csv)
else :
    df = pd.DataFrame(raw_pairs)

df["support_doc_ids"]  = df["question"].map(lambda q: lookup[q]["support_doc_ids"])

In [ ]:
df.to_csv(csv, index=False)
print(f"Fichier CSV enregistré : {csv_name}")

In [ ]:
print("Exécution du chatbot pour générer les prédictions...")
chat_history = []
predictions, latencies = [], []
for row in tqdm(df.itertuples(), total=len(df)):
        if use_rag:
                start = time.time()
                result = rag_get_response(row.question)
        else :
                start = time.time()
                result = no_rag_get_response(row.question)
        latencies.append(time.time() - start)
        predictions.append(result)

In [ ]:
df['prediction'] = predictions
df['latency'] = latencies

In [ ]:
# Save results to CSV
df.to_csv(csv_name, index=False, encoding="utf-8")
print(f"\nRésultats sauvegardés dans : {csv_name}")

In [ ]:
print("Configuration des prompts d'évaluation...")
class EvalNote(TypedDict):
    note:        Annotated[float,  "de 0 à 5"]
    explication: Annotated[str,    "raisonnement concis"]

model_strict = model.with_structured_output(EvalNote, strict=True)


correctness_prompt = PromptTemplate.from_template("""
Tu es un professeur qui note la justesse factuelle.
Donne une note de 0 (totalement faux) à 5 (parfaitement correct).
Réponds **obligatoirement** au format JSON suivant :

{{
  "note": <nombre>,
  "explication": "<justification courte>"
}}

QUESTION :
{question}

RÉPONSE ATTENDUE :
{reference}

RÉPONSE DU CHATBOT :
{prediction}

JSON :
""")

relevance_prompt = PromptTemplate.from_template("""
Tu es un professeur. Donne une note de 0 (hors-sujet) à 5 (parfaitement pertinent).
Réponds **obligatoirement** au format JSON suivant :

{{
  "note": <nombre>,
  "explication": "<justification courte>"
}}

QUESTION :
{question}

RÉPONSE DU CHATBOT :
{prediction}

JSON :
""")

faithfulness_prompt = PromptTemplate.from_template("""
Tu es un évaluateur. Les informations de la réponse proviennent-elles bien des documents ?
Donne une note de 0 (hallucination complète) à 5 (entièrement fondé sur le contexte).
Réponds **obligatoirement** au format JSON suivant :

{{
  "note": <nombre>,
  "explication": "<justification courte>"
}}
                                                   
DOCUMENTS :
{docs}

RÉPONSE DU CHATBOT :
{prediction}

JSON :
""")

In [ ]:
print("Initialisation des chaînes LLM pour l'évaluation...")
correctness_chain  = correctness_prompt  | model_strict
relevance_chain    = relevance_prompt    | model_strict
faithfulness_chain = faithfulness_prompt | model_strict

In [ ]:
print("Évaluation des réponses du chatbot...")
notes_correct, notes_relev, notes_faith = [], [], []
exps_correct,  exps_relev,  exps_faith  = [], [], []

for row in tqdm(df.itertuples(), total=len(df), desc="Évaluation"):

    docs = vector.similarity_search(row.question, k=4)
    ctx  = "\n\n".join(d.page_content for d in docs)
    while True:
        try : 
            # --- Correctness
            res = correctness_chain.invoke({
                "question":  row.question,
                "reference": row.answer,
                "prediction":row.prediction,
            })
            break
        except Exception as e:
            print(f"Erreur lors de l'appel à l'API : {e}")
    notes_correct.append(res["note"]) 
    exps_correct.append(res["explication"])  
    while True:
        try : 
            # --- Relevance
            res = relevance_chain.invoke({
                "question":  row.question,
                "prediction":row.prediction,
            })
            break
        except Exception as e:
            print(f"Erreur lors de l'appel à l'API : {e}")
    notes_relev.append(res["note"])
    exps_relev.append(res["explication"])
    while True:
        try : 
            # --- Faithfulness / groundedness
            res = faithfulness_chain.invoke({
                "docs":       ctx,
                "prediction": row.prediction,
            })
            break
        except Exception as e:
            print(f"Erreur lors de l'appel à l'API : {e}")
    notes_faith.append(res["note"])
    exps_faith.append(res["explication"])

In [ ]:
df["note_correct"]       = notes_correct
df["note_relevance"]     = notes_relev
df["note_faithfulness"]  = notes_faith

df["explication_correct"]      = exps_correct
df["explication_relevance"]    = exps_relev
df["explication_faithfulness"] = exps_faith

df.to_csv(csv, index=False, encoding="utf-8")

In [ ]:
sims = []
st_model = SentenceTransformer("all-MiniLM-L6-v2")
for row in tqdm(df.itertuples(), total=len(df), desc="Évaluation"):
    sims.append(util.cos_sim(
        st_model.encode(row.prediction, convert_to_tensor=True),
        st_model.encode(row.answer,     convert_to_tensor=True)
    ).item())

In [ ]:
# Store results
df["semantic_sim"] = sims

In [ ]:
# Save results to CSV
df.to_csv(csv, index=False, encoding="utf-8")
print(f"\nRésultats sauvegardés dans : {csv}")

In [ ]:
print("\n\nRésultats de l'évaluation :\n")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)
display(df)